IMPORT

In [1]:
import requests
import json
import pandas as pd
import mysql.connector
import sys
import os

CONNECTING TO DATABASE

In [2]:
mydb = mysql.connector.connect(
host="localhost",
user="admin",
password="admin",
database='cricinfo',
)
mycursor = mydb.cursor()

INPUT

In [4]:
team1_id = int(input("Enter the first team ID: "))
team1 = []
for i in range(1):
    player = int(input("Enter the player ID of Team 1: "))
    team1.append(player)

team2_id = int(input("Enter the second team ID: "))
team2 = []
for i in range(1):
    player = int(input("Enter the player ID of Team 2: "))
    team2.append(player)

match_type = int(input("Enter the match type: "))

ground_id = int(input("Enter the ground ID: "))

batting_first = int(input("Enter the batting first team (1 or 2): "))
bowling_first = int(input("Enter the bowling first team (1 or 2): "))
if batting_first == 1:
    team1_innings = 1
    team2_innings = 2
else:
    team1_innings = 2
    team2_innings = 1

FETCHING REQUIRED DATA

In [ ]:
# Fetching data for all 22 players
player_data_all_format = []
player_data_match_format = []
player_data_ground = []
player_data_opponent = []
player_data_innings = []
player_data_all = []

# Fetching data based on all formats
team = [379140]
for player_id in team1 + team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s"""
    mycursor.execute(query,(player_id,))
    result = mycursor.fetchone()
    player_data_all_format.append(result[0])

# Fetching data based on match format
for player_id in team1 + team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Match_Type = %s"""
    mycursor.execute(query,(player_id,match_type))
    result = mycursor.fetchone()
    player_data_match_format.append(result[0])

# Fetching data based on ground
for player_id in team1 + team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Ground_ID = %s"""
    mycursor.execute(query,(player_id,ground_id))
    result = mycursor.fetchone()
    player_data_ground.append(result[0])

# Fetching data based on opponent
for player_id in team1:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Opponent_Team_ID = %s"""
    mycursor.execute(query,(player_id,team2_id))
    result = mycursor.fetchone()
    player_data_opponent.append(result[0])

for player_id in team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Opponent_Team_ID = %s"""
    mycursor.execute(query,(player_id,team1_id))
    result = mycursor.fetchone()
    player_data_opponent.append(result[0])

# Fetching data based on innings
for player_id in team1:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Innings_Order = %s"""
    mycursor.execute(query,(player_id,team1_innings))
    result = mycursor.fetchone()
    player_data_innings.append(result[0])

for player_id in team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Innings_Order = %s"""
    mycursor.execute(query,(player_id,team2_innings))
    result = mycursor.fetchone()
    player_data_innings.append(result[0])

# Fetching data based on match format, ground, opponent, and innings combined
for player_id in team1:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Match_Type = %s AND Ground_ID = %s AND Opponent_Team_ID = %s AND Innings_Order = %s"""
    mycursor.execute(query,(player_id,match_type,ground_id,team2_id,team1_innings))
    result = mycursor.fetchone()
    player_data_all.append(result[0])

for player_id in team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Match_Type = %s AND Ground_ID = %s AND Opponent_Team_ID = %s AND Innings_Order = %s"""
    mycursor.execute(query,(player_id,match_type,ground_id,team1_id,team2_innings))
    result = mycursor.fetchone()
    player_data_all.append(result[0])

In [3]:
df = pd.read_excel('dataset.xlsx')
print(df.head())

   Player_ID  Team_ID  Opponent_Team_ID  Ground_ID  Match_Date  Innings_Order  \
0     219889        2                 7      56336  2017-01-13              1   
1     227758        7                 2      56336  2017-01-13              2   
2     227762        7                 2      56336  2017-01-13              2   
3     230193        2                 7      56336  2017-01-13              1   
4     267192        2                 7      56336  2017-01-13              1   

   Match_Type  Matches  In_Dream_Team  
0           2        1              0  
1           2        1              1  
2           2        1              0  
3           2        1              1  
4           2        1              0  
